In [3]:
import numpy as np
import sys
import os
import traceback
import itertools
import pickle
import time

import scipy.weave as weave
from scipy.weave import converters

from sigvisa import Sigvisa
from sigvisa.infer.propose_hough_new import visualize_hough_array, template_origin_times, template_amp_transfer

In [4]:
lonbins = 60
latbins = 120
bin_width_deg = 180/lonbins
stime = 1238887580.0
time_tick_s = 50
timebins = 144 # 7200/time_tick_s
min_mb = 3.0
max_mb = 8.0
mbbins = 10
mb_bin_width = (max_mb-min_mb)/mbbins
max_depth = 700
bin_width_km = 40000/latbins
depthbins = int(np.ceil(max_depth / bin_width_km))
depthbin_width_km = max_depth/float(depthbins)

sites = lonbins*latbins*depthbins
bins_at_site = timebins * mbbins
print sites, bins_at_site

14400 1440


In [5]:
sfile = "/home/dmoore/python/sigvisa/logs/mcmc/02135/step_000000/pickle.sg"
with open(sfile, 'rb') as f:
    sg = pickle.load(f)
print "read sg"
fitz_wn = None
for wn in sg.station_waves["FITZ"]:
    fitz_wn = wn
    
atimes = []
amps = []
for eid, phase in wn.arrivals():
    v, _ = wn.get_template_params_for_arrival(eid, phase)
    atimes.append(v['arrival_time'])
    amps.append(v['coda_height'])
    

read sg


/home/dmoore/python/sigvisa/models/signal_model.py:581: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return TransientCombinedSSM(components, TSSM_NOISE_PADDING)


In [6]:
array = np.zeros((lonbins, latbins, depthbins, timebins, mbbins), dtype=np.float32)

In [7]:
s = Sigvisa()
sta = "FITZ"
phaseids = (1,5)




In [8]:
def generate_sta_hough(sta_hough_array, atimes, amps, ttimes, amp_transfer, amp_transfer_stds, 
                       stime, time_tick_s, bin_width_deg):
    lonbins, latbins, depthbins, timebins, mbbins = sta_hough_array.shape
    time_radius = time_tick_s / 2.0
    lonbin_deg = 360.0/lonbins
    
    ntemplates = len(atimes)

    amps = np.asarray(amps, dtype=np.float)
    atimes = np.asarray(atimes, dtype=np.float)
    
    detection_prob = 0.8 # TODO calibrate by magnitude, phaseid
    uniform_atime = 1.0/3600
    uniform_amp_prior = 1.0/10
    additional_event_prob = 0.6
    background_cost_lp = float(np.log((1-detection_prob) * uniform_atime * uniform_amp_prior * additional_event_prob))
    detection_lp = float(np.log(detection_prob))
    print "background cost lp", background_cost_lp, "detection lp", detection_lp
    
    cdfs = """
    // cdf of Laplace(0.0, 5.0) computed for integer x values [-40, 40] inclusive
    double laplace_table[] ={0.00017, 0.00020, 0.00025, 0.00031, 0.00037, 0.00046, 0.00056, 0.00068, 0.00083, 0.00101, 0.00124, 0.00151, 0.00185, 0.00226, 0.00276, 0.00337, 0.00411, 0.00503, 0.00614, 0.00750, 0.009\
16, 0.01119, 0.01366, 0.01669, 0.02038, 0.02489, 0.03041, 0.03714, 0.04536, 0.05540, 0.06767, 0.08265, 0.10095, 0.12330, 0.15060, 0.18394, 0.22466, 0.27441, 0.33516, 0.40937, 0.50000, 0.59063, 0.66484, 0.72\
559, 0.77534, 0.81606, 0.84940, 0.87670, 0.89905, 0.91735, 0.93233, 0.94460, 0.95464, 0.96286, 0.96959, 0.97511, 0.97962, 0.98331, 0.98634, 0.98881, 0.99084, 0.99250, 0.99386, 0.99497, 0.99589, 0.99663, 0.9\
9724, 0.99774, 0.99815, 0.99849, 0.99876, 0.99899, 0.99917, 0.99932, 0.99944, 0.99954, 0.99963, 0.99969, 0.99975, 0.99980, 0.99983};
    // cdf of Normal(0, 1) computed for 81 values [-4, 4] inclusive
    double gaussian_table[]={0.00003, 0.00005, 0.00007, 0.00011, 0.00016, 0.00023, 0.00034, 0.00048, 0.00069, 0.00097, 0.00135, 0.00187, 0.00256, 0.00347, 0.00466, 0.00621, 0.00820, 0.01072, 0.01390, 0.01786, 0.02275, 0.02872, 0.03593, 0.04457, 0.05480, 0.06681, 0.08076, 0.09680, 0.11507, 0.13567, 0.15866, 0.18406, 0.21186, 0.24196, 0.27425, 0.30854, 0.34458, 0.38209, 0.42074, 0.46017, 0.50000, 0.53983, 0.57926, 0.61791, 0.65542, 0.69146, 0.72575, 0.75804, 0.78814, 0.81594, 0.84134, 0.86433, 0.88493, 0.90320, 0.91924, 0.93319, 0.94520, 0.95543, 0.96407, 0.97128, 0.97725, 0.98214, 0.98610, 0.98928, 0.99180, 0.99379, 0.99534, 0.99653, 0.99744, 0.99813, 0.99865, 0.99903, 0.99931, 0.99952, 0.99966, 0.99977, 0.99984, 0.99989, 0.99993, 0.99995, 0.99997};

    double laplace_cdf(double x) {
        int ix = (int) floor(x);
        int ix2 = ix+1;
        if (ix2 <= -40) return 0.0;
        if (ix >= 40) return 1.0;
        double tbl1 = laplace_table[ix+40];
        double tbl2 = laplace_table[ix2+40];
        double y = x-ix;
        return (1-y)*tbl1 + y*tbl2;
    }
    
    double gaussian_cdf(double x) {
        int ix = (int) floor(x*10);
        int ix2 = ix+1;
        if (ix2 <= -40) return 0.0;
        if (ix >= 40) return 1.0;
        double tbl1 = gaussian_table[ix+40];
        double tbl2 = gaussian_table[ix2+40];
        double y = x-ix;
        return (1-y)*tbl1 + y*tbl2;
    }
    """

    # ttimes, amp_transfers, amp_transfer_stds: loc bins
    # phase_score: timebin*mbbin
    # assoc: int, nphases*timebin*mbbin
    # atimes, amps
    # background_cost_lp
    # detection
    
    nphases = ttimes.shape[3]
    phase_score = np.empty((timebins, mbbins))
    assoc = np.empty((nphases, timebins, mbbins), dtype=np.uint8)
    
    code = """

for (int i=0; i < lonbins; ++i) {
    for (int j=0; j < latbins; ++j) {
        for (int k=0; k < depthbins; ++k) {
            for (int phaseidx=0; phaseidx < nphases; ++phaseidx) {
                double ttime = ttimes(i,j, k, phaseidx);
                double amp_transfer = amp_transfers(i, j, k, phaseidx);
                double amp_transfer_std = amp_transfer_stds(i, j, k, phaseidx);
                
                // TODO use memset
                for (int timebin=0; timebin < timebins; ++timebin) {
                    for (int mbbin=0; mbbin < mbbins; ++mbbin) {
                        phase_score(timebin, mbbin) = background_cost_lp;
                        assoc(phaseidx, timebin, mbbin) = -1;
                    }
                }
                
                for (int t=0; i < ntemplates; ++t) {
                    double atime = atimes(t);
                    double amp = amps(t);
                    double origin_time = atime - ttime;
                    double mb = amp - amp_transfer;
                
                    if (origin_time < stime  || origin_time > stime + timebins*time_tick_s) { continue; }

                    int min_plausible_timebin = std::max(0, int((origin_time - 30.0-stime) / time_tick_s));
                    int max_plausible_timebin = std::min(timebins-1, int((origin_time + 30.0-stime) / time_tick_s));
                    int min_plausible_mbbin = std::max(0, int((mb-3*amp_transfer_std - min_mb) / mb_bin_width) );
                    int max_plausible_mbbin = std::min(mbbins-1, int((mb+3*amp_transfer_std - min_mb) / mb_bin_width) );

                    // every bin should track p(event | best assoc) and idx(best assoc)
                    // for each phase, this means we track the "score" of the best assoc. 
                    // there is a default score corresponding to the "none" assoc. 
                    for (int timebin=min_plausible_timebin; timebin <= max_plausible_timebin; timebin++) {
                        // p(atime | bin)
                        double timebin_left = (stime + timebin * time_tick_s) - origin_time;
                        double timebin_right = timebin_left + time_tick_s;
                        double bin_weight = laplace_cdf(timebin_right) - laplace_cdf(timebin_left);
                        double atime_lp = log(bin_weight) - log(time_tick_s);
                        
                        for (int mbbin = min_plausible_mbbin; mbbin <= max_plausible_mbbin; mbbin++) {
                        
                            // don't allow templates that have previously 
                            // been associated with other phases.
                            bool exclude_tmpl_from_bin = 0;
                            for (int oldphase = 0; oldphase < phaseidx; ++oldphase) {
                                if (assoc(phaseidx, timebin, mbbin) == t) {
                                    exclude_tmpl_from_bin = 1;
                                    break;
                                }
                            }
                            if (exclude_tmpl_from_bin) { continue; }
                        
                            double mb_left = min_mb + mbbin*mb_bin_width;
                            double amp_transfer_left = amp - mb_left;
                            double at_residual_left =  amp_transfer_left - amp_transfer;
                            double at_residual_right = at_residual_left + mb_bin_width;
                            //double bin_weight = gaussian_cdf(at_residual_right/amp_transfer_std) 
                            //                       - gaussian_cdf(at_residual_left/amp_transfer_std);
                            double bin_weight = 1.0;
                            double mb_lp = log(bin_weight) - log(mb_bin_width);
                            double tmpl_score = detection_lp + atime_lp + mb_lp;
                            
                            double oldval = phase_score(phaseidx,timebin, mbbin);
                            if (tmpl_score > oldval) {
                                phase_score(timebin, mbbin) = tmpl_score;
                                assoc(phaseidx, timebin, mbbin) = t;
                            }

                        }
                    }
                }
            }
        }
    }
}
    """
    weave.inline(code,['latbins', 'lonbins', 'depthbins', 'timebins', 'mbbins', 'nphases', 
                       'sta_hough_array', 'ttimes', 'amp_transfers', 'amp_transfer_stds', 
                       'phase_score', 'assoc', 'bin_width_deg', 'time_tick_s', 'mb_bin_width', 
                       'bin_width_km', 'stime', 'min_mb', 'background_cost_lp', 'detection_lp', 
                       'amps', 'atimes', 'ntemplates'], 
                 support_code=cdfs, headers=["<math.h>", "<unordered_set>"], type_converters = converters.blitz,verbose=2,compiler='gcc', extra_compile_args=["-std=c++11"])
    #print "added template"

In [ ]:

ttimes = np.concatenate([-template_origin_times(sta, 0, phaseid, latbins, depthbins)[:,:,:,np.newaxis] for phaseid in phaseids], axis=3)

wn = sg.station_waves[sta][0]
atP, atvP = template_amp_transfer(sg, wn, "P", latbins=latbins, depthbins=depthbins)
atS, atvS = template_amp_transfer(sg, wn, "S", latbins=latbins, depthbins=depthbins)
amp_transfers = np.concatenate([atP[:,:,:,np.newaxis], atS[:,:,:,np.newaxis]], axis=3)
amp_transfer_stds = np.concatenate([np.sqrt(atvP)[:,:,:,np.newaxis], np.sqrt(atvS)[:,:,:,np.newaxis]], axis=3)


In [ ]:
generate_sta_hough(array, atimes, amps, ttimes, 
                   amp_transfers, amp_transfer_stds, 
                   sg.event_start_time, time_tick_s, bin_width_deg)

background cost lp -12.6115377536 detection lp -0.223143551314
<weave: compiling>
running build_ext
running build_src
build_src
building extension "sc_09b85b0ad03548eb04764fea1f0f2cb80" sources
build_src: building npy-pkg config files
customize UnixCCompiler
customize UnixCCompiler using build_ext
customize UnixCCompiler
customize UnixCCompiler using build_ext
building 'sc_09b85b0ad03548eb04764fea1f0f2cb80' extension
compiling C++ sources
C compiler: g++ -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -fPIC

compile options: '-I/home/dmoore/.virtualenvs/sigvisa/local/lib/python2.7/site-packages/scipy/weave -I/home/dmoore/.virtualenvs/sigvisa/local/lib/python2.7/site-packages/scipy/weave/scxx -I/home/dmoore/.virtualenvs/sigvisa/local/lib/python2.7/site-packages/scipy/weave/blitz -I/home/dmoore/.virtualenvs/sigvisa/local/lib/python2.7/site-packages/numpy/core/include -I/usr/include/python2.7 -c'
extra options: '-std=c++11'
g++: /home/dmoore/.virtualenvs/sigvisa/local/lib/python2.7/

[  3.16712418e-05   4.80963440e-05   7.23480439e-05   1.07799733e-04
   1.59108590e-04   2.32629079e-04   3.36929266e-04   4.83424142e-04
   6.87137938e-04   9.67603213e-04   1.34989803e-03   1.86581330e-03
   2.55513033e-03   3.46697380e-03   4.66118802e-03   6.20966533e-03
   8.19753592e-03   1.07241100e-02   1.39034475e-02   1.78644206e-02
   2.27501319e-02   2.87165598e-02   3.59303191e-02   4.45654628e-02
   5.47992917e-02   6.68072013e-02   8.07566592e-02   9.68004846e-02
   1.15069670e-01   1.35666061e-01   1.58655254e-01   1.84060125e-01
   2.11855399e-01   2.41963652e-01   2.74253118e-01   3.08537539e-01
   3.44578258e-01   3.82088578e-01   4.20740291e-01   4.60172163e-01
   5.00000000e-01   5.39827837e-01   5.79259709e-01   6.17911422e-01
   6.55421742e-01   6.91462461e-01   7.25746882e-01   7.58036348e-01
   7.88144601e-01   8.15939875e-01   8.41344746e-01   8.64333939e-01
   8.84930330e-01   9.03199515e-01   9.19243341e-01   9.33192799e-01
   9.45200708e-01   9.55434537e-01